# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model

In [2]:
# Hyperparameter
input_size = 784 # MNIST: 28x28 = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5 

# Create the model
class NN(nn.Module):  # we inherit our model class from nn.module
    def __init__(self, input_size, num_classes):  
        super(NN, self).__init__()  # The super calls the initialization method of the parent class
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load Data

In [7]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Training

In [4]:
# Intialize NN
# model = CNN(input_size=input_size, num_classes=num_classes).to(device)
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the NN
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to device
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get the correct shape
        data = data.reshape(data.shape[0], -1)

        # Forward propagation
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward propagation
        optimizer.zero_grad() # initialize all gradients to zero for each batch
        loss.backward()

        # Gradient descent or Adam step
        optimizer.step()

# Performance

In [5]:
# Check accuracy on training and test sets
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)  # scores is 64x10 and we want to know which one of those the is the maximum value, so in max: dim=1
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    print(f'got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

In [6]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
got 58389 / 60000 with accuracy 97.31
Checking accuracy on test data
got 9645 / 10000 with accuracy 96.45
